# notebook for doing sic with $n=30$

In [214]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [215]:
import numpy as np
import matplotlib.pyplot as plt
from utils import calculate_rate, create_model
import os
import json
from types import SimpleNamespace
import tensorflow as tf



In [216]:
n=30
L=50
snr_set = np.linspace(-1.5,3,4)

In [217]:
exp = 'mac_three_users_n30_fp'
data_dir = '/scratch/users/cmatson/nn-codes/data/'
experiment_dir = os.path.join(data_dir, f'experiment_{exp}')

In [218]:
experiment_set = []
mac_set=[]
test = 0
for snr1 in snr_set:
    k1, _, r1, c1 = calculate_rate(k=None, n=n, SNR=snr1)
    experiment_set.append((test, k1, snr1, snr1, 1)) # test, k, snr, snr_eff, h
    i1=test
    test+=1
    
    for snr2 in snr_set:
        N0 = 10**(-snr1/10)

        # calculate the scale factor and effective snr for U2 st we have same N0
        h2 = N0*10**(snr2/10)
        snr2_eff = 10*np.log10(h2/(1+N0))
        
        k2, _, r2, c2 = calculate_rate(k=None, n=n, SNR=snr2_eff)
        if k2==0:
            continue
        experiment_set.append((test, k2, snr2, snr2_eff, h2))
        i2=test
        test+=1

        for snr3 in snr_set:
            # calculate the scale factor and effective snr for U3 st we have same N0
            h3 = N0*10**(snr3/10)
            snr3_eff = 10*np.log10(h3/(1+h2+N0))
        
            k3, _, r3, c3 = calculate_rate(k=None, n=n, SNR=snr3_eff)
            if k3==0:
                continue
            experiment_set.append((test, k3, snr3, snr3_eff, h3))
            i3=test
            test+=1
            
            
            mac_set.append((i1, i2, i3))

### check the training

In [219]:
didnot = []
performance_ber = []
performance_gp = []
for test, (t, k, snr, snr_eff, h) in enumerate(experiment_set):
    if not os.path.exists(os.path.join(experiment_dir, f'test_{test}', 'val_history.npy')):
        print(f' {test} did not complete')
        didnot.append(test)
        continue
    hv = np.load(os.path.join(experiment_dir, f'test_{test}', 'val_history.npy'), allow_pickle=True).item()
    
        
    performance_ber.append(1-max(hv['binary_accuracy']))
    performance_gp.append((k/n)*max(hv['binary_accuracy']))


    
    if False:
        fig, ax = plt.subplots(1,2, tight_layout=True)
        ax[0].plot(hv['loss'], label='loss')
        ax[0].set_xlim([0,101])
        ax[0].set_title('loss')

        ax[1].plot(1-np.array(hv['binary_accuracy']), 'r-', label='acc')
        ax[1].set_xlim([0,101])
#         ax[1].set_yscale('log')
        ax[1].set_ylim([0,.15])
        ax[1].set_title('acc')

        fig.suptitle(f'{test}: {k}/{n} @ snr {snr_eff:.1f}')
performance_ber = np.array(performance_ber)
performance_gp = np.array(performance_gp)

### do the mac tests

In [220]:
def get_model(test, get_ds=False):
    with open(os.path.join(experiment_dir, f'test_{test}', 'args.json')) as f:
            args = json.load(f)
            
    args = SimpleNamespace(**args)
    m = create_model(args=args, load_weights=True)
    
    m.compile(optimizer=tf.optimizers.Adam(),
          loss=tf.losses.BinaryCrossentropy(from_logits=False),
          metrics=tf.metrics.BinaryAccuracy())


    if get_ds:
        ds = create_dataset(args.k, L)
        return m, ds, args
    
    return m, args
    

In [221]:
os.environ['NN_CODE_ENV'] = 'm2'

In [222]:
from analyze_three_users_script_v2 import perform_sic

In [223]:
iterations_set = [1, 2, 5, 10, 50, 100]
# iterations_set = [1,2]

In [224]:
RECALC=False
if RECALC:
    sic3_data = np.zeros((len(mac_set), len(iterations_set), 4, 3)) # mac_test, iterations, (it, ber, gp, mse), 3 users
    sic2_data = np.zeros((len(mac_set), len(iterations_set), 4, 2)) # mac_test, iterations, (it, ber, gp, mse), 3 users


    # for each mac experiment
    for mac_test, (i1, i2, i3) in enumerate(mac_set):
        # create the models and load the weights
        m1, args1 = get_model(i1)
        m2, args2 = get_model(i2)
        m3, args3 = get_model(i3)

        # construct the params for sic
        k1 = args1.k
        k2 = args2.k
        k3 = args3.k

        #test, k, snr, snr_eff, h
        snr1 = args1.snr
        snr2_eff=args2.snr
        snr3_eff=args3.snr
        snr2=experiment_set[i2][2]
        snr3=experiment_set[i3][2]
        h2 = experiment_set[i2][-1]
        h3 = experiment_set[i3][-1]

        sic_params = {'k1':k1, 'k2':k2, 'k3':k3, 'n':n, 'L':100,
                      'snr1':snr1, 'h2':h2, 'h3':h3}

        test_name = f'{mac_test}_{snr1}_{snr2}_{snr3}'.replace('-','n').replace('.','p')
        print(f'mac3 test {mac_test} snrs ({snr1}, {snr2}, {snr3}) (single user tests {i1},{i2},{i3})')

        # only do snr1!=snr2
        if mac_test not in np.arange(0,len(mac_set),4):
            continue
        
        # do iterations on three user mac
        for it, iterations in enumerate(iterations_set):
            print(f' {iterations} iterations')
            mac3_ber, mac3_gp, mac3_mse = perform_sic(m1, m2, m3, sic_params, iterations=iterations)
            sic3_data[mac_test, it] = [[iterations]*3, mac3_ber, mac3_gp, mac3_mse]
            print(f'  3 user ber: {mac3_ber[0]:.3f}, {mac3_ber[1]:.3f}, {mac3_ber[2]:.3f}')

            mac2_ber, mac2_gp, mac2_mse = perform_sic(m1, m2, -1, sic_params, iterations=iterations)
            sic2_data[mac_test, it] = [[iterations]*2, mac2_ber, mac2_gp, mac2_mse]
            print(f'  2 user ber: {mac2_ber[0]:.3f}, {mac2_ber[1]:.3f}')


        print('')


In [225]:
if RECALC:
    np.save(os.path.join(experiment_dir, f'sic3_n{n}_data.npy'), sic3_data)
    np.save(os.path.join(experiment_dir, f'sic2_n{n}_data.npy'), sic2_data)


else:
    sic3_data = np.load(os.path.join(experiment_dir, f'sic3_n{n}_data.npy'))
    sic2_data = np.load(os.path.join(experiment_dir, f'sic2_n{n}_data.npy'))